In [ ]:
%matplotlib inline

import random
import os
import sys
import csv
import numpy as np
import pandas as pd
import matplotlib

from matplotlib import pyplot as plt
from glob import glob

from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf 
from tensorflow.keras.applications import DenseNet121, VGG16, VGG19, InceptionV3, ResNet50
from tensorflow.keras.layers import Input, Dense, Lambda, Activation, concatenate, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model

In [ ]:
from MyDirectoryIterator import MyDirectoryIterator

In [ ]:
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300

# Prepare dataset for training, tuning and testing

## Initialize Parameters 

In [ ]:
COX_GROUP_SIZE = 32  # tile group size for compute cox loss
MODEL_BATCH_SIZE = 2  # batch size for model training

EPOCHS = 100
IMG_ORI_SIZE = 1196  # roi img size
IMG_SIZE = 224  # tile img size for network input
CHANNEL = 3 

In [ ]:
# constant from model training
RISK_MEDIAN = -0.6851 
RISK_MEAN = -0.6993
RISK_STD = 0.3014

In [ ]:
mode = "load"
model_h5_file = '../model/prognosis.h5'
model_h5_file = os.path.abspath(model_h5_file)
test_dir = '../data/prognosis'
metadata_path = '../data/metadata.csv'

## Data generator and data augment

### Load data

In [ ]:
def get_pid_by_filename(filename):
    return filename.split('_')[0][:12]

In [ ]:
def load_imgs(img_dir):
    filenames = glob(img_dir+'/*')
    pids = []
    for filename in filenames:
        pid = get_pid_by_filename(os.path.basename(filename)) 
        print(pid)
        pids.append(pid)
    return np.array(pids)

In [ ]:
# read surival and censor data for pids
censor_map = {"Dead": 0, "Alive": 1}
def load_metadata(path, pids):
    metadata = pd.read_csv(path)
    metadata_dict = dict()
    for _, row in metadata.iterrows():
        metadata_dict[row['TCGA ID']] = row
    
    survival = np.empty((len(pids), 1),  dtype='int32')
    censored = np.empty((len(pids), 1), dtype='int32')
    sur_map = {}
    cen_map = {}
    
    error = False
    for i, pid in enumerate(pids):
        try:
            sur_map[pid] = survival[i] = metadata_dict[pid]['survival']
            cen_map[pid] = censored[i] = censor_map[metadata_dict[pid]['censored']]
        except KeyError:
            error = True
            print("keyerror for", pid)
    if error:
        raise Exception("load metadata error")
        
    return survival, censored, sur_map, cen_map

In [ ]:
t_pids = load_imgs(test_dir)

In [ ]:
_, _, t_sur_map, t_cen_map = load_metadata(metadata_path, t_pids)

In [ ]:
# test set
t_pids = np.unique(t_pids) 
t_surs = np.array([t_sur_map[pid] for pid in t_pids])
t_cens = np.array([t_cen_map[pid] for pid in t_pids])

indices = np.argsort(t_surs)

t_pids = t_pids[indices]
t_surs = t_surs[indices]
t_cens = t_cens[indices]

### Build generator

# Build risk model with VGG19

In [ ]:
base_model = VGG19(weights=None, include_top=True)

In [ ]:
transfer_layer = base_model.get_layer('block5_pool')

conv_model = Model(inputs=base_model.input,
                   outputs=transfer_layer.output)

new_model = Sequential()

new_model.add(conv_model)
new_model.add(Flatten())
new_model.add(Dropout(0.3))
new_model.add(Dense(1))

risk_model = new_model

In [ ]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

# Build prognostic model with COX

In [ ]:
# Need not sort by survival, besause it've been done in data generator, which make the training more stable.
def compute_loss(risks, censored):   
    observed = 1 - censored
    
    # calc the cox negative log likelihood 
    risk_exp = tf.exp(risks)  # exp 
    
    partial_sum = tf.cumsum(risk_exp, axis=1, reverse=True)  # cumsum
    log_at_risk = tf.log(partial_sum)
    diff = risks - log_at_risk  # sub
    
    times = tf.multiply(diff, observed, name='times')  # deal with the censored
    loss = -tf.reduce_sum(times, axis=1, name='loss')
    return risks, risk_exp, partial_sum, log_at_risk, diff, times, loss

In [ ]:
def build_model(group_size):
    input_img_list = []
    for i in range(group_size):
        inp = Input(shape=(IMG_SIZE, IMG_SIZE, CHANNEL), name='input_img'+str(i))
        input_img_list.append(inp)
    
    input_censor = Input(shape=(group_size, ), name='input_censor')
    
    risks = [risk_model(img) for img in input_img_list]
    
    risk_layer = concatenate(risks, name='risk_concat')
      
    # build loss layer
    loss = Lambda(lambda x: compute_loss(*x))([risk_layer, input_censor])
    
    model = Model(inputs=[*input_img_list, input_censor], outputs=loss[6])  

    return model

In [ ]:
model = build_model(COX_GROUP_SIZE)

# Train model

## Predefined Fucntions

In [ ]:
def calc_cindex(risk, survival, censored):
    total = 0.0
    success = 0.0
    for i in range(len(survival)):
        for j in range(i + 1, len(survival)):
            if risk[i] == risk[j]:
                continue  
            if censored[i] == 0 and censored[j] == 0:
                total = total + 1
                if survival[i] > survival[j]:
                    if risk[j] > risk[i]:
                        success = success + 1
                elif survival[j] > survival[i]:
                    if risk[i] > risk[j]:
                        success = success + 1
                elif risk[i] == risk[j]:
                    success = success + 1
            elif censored[i] == 1 and censored[j] == 0:
                if survival[i] >= survival[j]:
                    total = total + 1
                    if risk[j] > risk[i]: 
                        success = success + 1
            elif censored[j] == 1 and censored[i] == 0:
                if survival[j] >= survival[i]:
                    total = total + 1
                    if risk[i] > risk[j]:
                        success = success + 1
    return success/total

In [ ]:
def get_level2_risk(t_risks, train_median):
    t_risks_level_2 = []
    for risk in t_risks:
        if risk <= train_median:
            t_risks_level_2.append(1)
        else:
            t_risks_level_2.append(2)

    return np.array(t_risks_level_2)

def calc_cindex_level_2(t_risks, t_surs, t_cens, train_median):
    t_risks_level_2 = get_level2_risk(t_risks, train_median)
    return calc_cindex(t_risks_level_2, t_surs, t_cens)

In [ ]:
def select(x, method='max', k=0):
    x_np = np.array(x)
    if method == 'avg':
        return np.average(x_np)
    elif method == 'med':
        return np.median(x_np)
    elif method == 'max':
        ind = np.argsort(x_np)[::-1]
        if k >= len(ind):
            k = len(ind)-1
        return x_np[ind[k]]

In [ ]:
def split_img(img, h_cnt, w_cnt, tar_size):
    imgs = []
    for i in range(0, h_cnt):
        for j in range(0, w_cnt):
            imgs.append(img[i*tar_size:(i+1)*tar_size, j*tar_size:(j+1)*tar_size, :])
    return np.array(imgs)

In [ ]:
IMG_SPLIT_H = 4
IMG_SPLIT_W = 4

def predict_whole_img(model, img, merge_method="avg"):
    imgs = split_img(img, IMG_SPLIT_H, IMG_SPLIT_W, IMG_SIZE)
    risks = model.predict(imgs)
    return np.average(risks)
    

In [ ]:
"""
    Predict risk of each pid in test_dir. 
    Each patient(pid) may have multiple ROI imgs, each img cut into 16 tile img, and their average risk as the risk of this picture.
    The risk of patient(pid) is the highest risk of all the pictures of this patient.
""" 
def predict_dir(model, test_dir, pids):
    risks =[]
    for pid in pids:
        p_risks = []
        for filename in glob(test_dir+'/*'+pid+'*'):
            img = img_to_array(load_img(filename))/255
            risk = predict_whole_img(model, img, merge_method="avg")
            p_risks.append(risk)
        risks.append(select(p_risks, method="max"))
    return np.array(risks)

In [ ]:
def compute_loss_test(risks, censored):
    risks = np.reshape(risks, -1)
    censored = np.reshape(censored, -1)
    
    observed = 1 - censored

    # calc the Cox negative log likelihood 
    risk_exp = np.exp(risks)  # exp 
    
    risk_exp = risk_exp[::-1]
    partial_sum = np.cumsum(risk_exp)
    partial_sum = partial_sum[::-1]
    
    log_at_risk = np.log(partial_sum)
    diff = risks - log_at_risk  # sub
    
    times = np.multiply(diff, observed)  # deal with the censored
    loss = -np.sum(times)
    
    return risks, risk_exp, partial_sum, log_at_risk, diff, times, loss

## Train the prognosis model

In [ ]:
risk_model.load_weights(model_h5_file)

# Test Model

## Predict 

In [ ]:
# _tr_risks = predict_dir(risk_model, train_dir, tr_pids)

In [ ]:
_t_risks = predict_dir(risk_model, test_dir, t_pids)

In [ ]:
print("risks_level_2 cindex:", calc_cindex_level_2(_t_risks, t_surs, t_cens, RISK_MEDIAN))

## Save result

In [ ]:
def print_results(filename, risks, train_median, train_mean, train_std, pids, cens, surs):
    risks_level_2 = get_level2_risk(risks, train_median)

    # zscore risks
    risks = (risks-train_mean) / train_std
    
    header = ("pid", "censored", "survival", "risks", "risks_level_2")
    data = list(zip(pids, cens, surs, risks, risks_level_2))

    print(header)
    with open(filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(header)
        for row in data:
            print(row)
            csv_writer.writerow(row)

In [ ]:
print_results('../result/result_progosis.csv', _t_risks, RISK_MEDIAN, RISK_MEAN, RISK_STD, t_pids, t_cens, t_surs)